# OSM Raster PolyPoints

This notebook contains analysis and visualizations for:

1. Loading polygons from masked raster 
2. Loading in points from OSM
3. Associating points with polygons

Updated: 2018-11-21

# Dependencies

In [56]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping

# Load in OSM and Polygons from csv

In [57]:
# will build out folders later

# data folder git will ignore
infilepath = "/home/cascade/tana-crunch-cascade/projects/NTL/data/" # git will ignore
outfilepath = "/home/cascade/tana-crunch-cascade/projects/NTL/temp_data/" # git will not ignore - NO BIG FILES 

# Local computer 
# infilepath = '/Users/cascade/Github/NTL/data/raw/worldpop/Africa-1km-Population/'
# outfilepath = '/Users/cascade/Github/NTL/temp_data/'

In [58]:
def load_points (file):
    """ This function loads a csv of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [113]:
# Load OSM Points
osm_point_gdf = load_points(outfilepath+'20181112_osm_africa_cities.csv')

In [115]:
osm_point_gdf.head(6)

,id,lat,lon,geometry
0,27564946,36.364519,6.608260,POINT (6.60826 36.364519)
1,27565103,35.703275,-0.649298,POINT (-0.6492975999999999 35.7032751)
2,253292160,36.713784,4.049392,POINT (4.0493919 36.7137843)
3,262963217,33.361177,6.860349,POINT (6.8603492 33.3611766)
4,262964638,33.109897,6.066102,POINT (6.066102 33.1098968)
5,264573224,33.806352,2.880862,POINT (2.8808616 33.8063518)


In [101]:
# Load masked polygons 
WP2000_poly = gpd.read_file(outfilepath+"20181121_poly_cities_africa2000mask600.shp")

### NEED TO FIND AWAY TO ASSOCIATE RASTER PIXELS AND POINTS WITH COUNTRIES Before we run giant for loop - show kelly problems in QGIS

1. Can likely clip points by polygon geometry and chunk
https://www.earthdatascience.org/courses/earth-analytics-python/spatial-data-vector-shapefiles/clip-vector-data-in-python-geopandas-shapely/

2. Can likely clip polygons by countries

https://gis.stackexchange.com/questions/168266/pyqgis-a-geometry-intersectsb-geometry-wouldnt-find-any-intersections



In [112]:
# Load africa countries -- 762 polygons because of islands 
# Africa_poly = gpd.read_file(outfilepath+"Africa_polys_test.shp")
# len(Africa_poly)

762

In [118]:
print(len(osm_point_gdf))
print(len(WP2000_poly))`


872
26973


# Function for searching if a point is within a polygon

In [119]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Old Code

In [ ]:
# building a function to check if points are in poly for lists of poly and points
# needs geopandas data frame with point and poly geometry 

# def poly_point (poly, point):
#     """
#     This function will check if points are inside polygons if given two gpds with points and polygons
#     Returns city names or no list 
#     """
    
#     out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
#     for index_point, row_point in point.iterrows():
#         for index_poly, row_poly in poly.iterrows():
#             if row_point['geometry'].within(row_poly['geometry']):
#                 country = row_point['Country']
#                 city = row_point['City']
#                 point_id = row_point['Id']
#                 point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
#                 poly_id = index_poly
#                 poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
#                 out_arr.append((country, 
#                                 city, 
#                                 point_id, 
#                                 point_geom, 
#                                 poly_id, 
#                                 poly_geom))
# #             else:
# #                 test.append('no')
#     return out_arr